In [1]:
import json
# from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
# import sqlite3
import http.client
import time
import collections
from collections import defaultdict

### Create Tables (run once)

In [2]:
data_folder = '../../components/Tools/NASHBoard/Data/'

#### Run the below scripts at the start of every week to update data into database

Connect to sportradar API

In [7]:
# testing API with python
f = open('../api_keys.json')
data = json.load(f)
API_key = data["SportRadar"]

# establish connection to API
conn = ''
def get_conn():
    global conn 
    conn = http.client.HTTPSConnection("api.sportradar.us")
get_conn()

# function to get and minimally format data given url
def get_data(url):
    conn.request("GET", url)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))

#### Get INJURIES across league

In [9]:
injury_data = get_data("/nba/trial/v7/en/league/injuries.json?api_key=%s" %API_key)
# reformat injuries data
injuries_list = []
i = 0
for row in injury_data['teams']:
    team = row['market'] + ' ' + row['name']
    for player in row['players']:
        injured = {}
        injured['id'] = i
        injured['team_name'] = team
        injured.update({k: player[k] for k in ('full_name', 'primary_position','reference')})
        injured.update({k: player['injuries'][0][k] for k in ('comment', 'status','start_date','update_date')})
        injured['player_id'] = injured.pop('reference')
        injuries_list.append(injured)
        i = i + 1
print(injuries_list[:2])

[{'id': 0, 'team_name': 'Washington Wizards', 'full_name': 'Daniel Gafford', 'primary_position': 'PF', 'comment': "Gafford left Monday's (Jan. 9) game with a right ankle sprain according to coach Wes Unseld Jr.", 'status': 'Day To Day', 'start_date': '2023-01-09', 'update_date': '2023-01-09', 'player_id': '1629655'}, {'id': 1, 'team_name': 'Washington Wizards', 'full_name': 'Bradley Beal', 'primary_position': 'SG', 'comment': 'The Wizards announced that Beal has a low-grade strain and will be re-evaluated in one week.', 'status': 'Out', 'start_date': '2023-01-03', 'update_date': '2023-01-05', 'player_id': '203078'}]


In [10]:
# store in json
with open(data_folder + 'injuries.json', 'w') as outfile:
    json.dump(injuries_list, outfile)

Get Season Schedules

In [44]:
# DONE FOR 2019-2021, only re-run for current year (below)
seasons = ['2019','2020','2021','2022']
season_schedules ={}
for season in seasons:
    season_schedules[season] = get_data("/nba/trial/v7/en/games/" + season + "/REG/schedule.json?api_key=%s" %API_key)
    # conn.request("GET", "/nba/trial/v7/en/games/" + season + "/REG/schedule.json?api_key=%s" %API_key)
    time.sleep(2)
    # res = conn.getresponse()
    # data = res.read()
    # season_schedules[season] = data.decode("utf-8")

In [58]:
# see below for current year
for season in seasons[3:]:
    print('season',season)
    games_list = []
    future_games_list = []
    for row in json.loads(season_schedules[season])['games']:
        game = {}
        if row['status'] == 'postponed' or row['status'] == 'cancelled':
            continue
        else:
            game['time_zone'] = row['time_zones']['venue']
            game['network'] = row['broadcasts'][0]['network'] if row.get('broadcasts') else 'missing'
            try:
                game['locale'] = row['broadcasts'][0]['locale']
            except:
                game['locale'] = 'home'
            game['home'] = row['home']['name']
            game['away'] = row['away']['name']
            if row['status'] == 'scheduled':
                game.update({k: row[k] for k in ('id', 'scheduled')})
                future_games_list.append(game)
            else:
                # try:
                game.update({k: row[k] for k in ('id', 'scheduled','home_points','away_points')})
                
                games_list.append(game)
        
    # store in json
    with open(data_folder + 'season_' + season + '_schedule.json', 'w') as outfile:
        json.dump(games_list, outfile)
    if season == '2022':
        with open(data_folder + 'season_' + season + 'remaining_schedule.json', 'w') as outfile:
            json.dump(future_games_list, outfile)

season 2022


In [13]:
# add weekly game to current season and remove from remaining games
season = "2022"
get_conn()
curr_season_schedule = get_data("/nba/trial/v7/en/games/" + season + "/REG/schedule.json?api_key=%s" %API_key)
games_list = []
future_games_list = []
num_games = len(curr_season_schedule['games'])
for i, row in enumerate(curr_season_schedule['games']):
    if i % 50 == 0:
        print('game',i,'of',num_games)
        
    game = {}
    if row['status'] == 'postponed' or row['status'] == 'cancelled':
        continue
    else:
        game['time_zone'] = row['time_zones']['venue']
        game['network'] = row['broadcasts'][0]['network'] if row.get('broadcasts') else 'missing'
        try:
            game['locale'] = row['broadcasts'][0]['locale']
        except:
            game['locale'] = 'home'
        game['home'] = row['home']['name']
        game['away'] = row['away']['name']
        # add to future games list if not yet played
        if row['status'] == 'scheduled':
            game.update({k: row[k] for k in ('id', 'scheduled')})
            future_games_list.append(game)
        # else, add to played games list
        else:
            # try:
            game.update({k: row[k] for k in ('id', 'scheduled','home_points','away_points')})
            games_list.append(game)
    
    # add to current year schedule jsons
    with open(data_folder + 'season_' + season + '_schedule.json', 'w') as outfile:
        json.dump(games_list, outfile)
    with open(data_folder + 'season_' + season + 'remaining_schedule.json', 'w') as outfile:
        json.dump(future_games_list, outfile)

WIP: 
Get game box scores for all games played (only shows team stats and leaders...)

In [9]:
# load all games
with open(data_folder + 'season_2022_schedule.json', "r") as read_content:
    season_2022 = json.load(read_content)
season_2022_ids = [game['id'] for game in season_2022]
season_2022_ids[:2]

['9f01b268-29c5-4f0e-bf67-21e3dbcf3005',
 'c665e441-9f38-48a7-8796-1f7292f97db1']

In [10]:
len(season_2022_ids)

376

In [159]:
# get team ids
get_conn()
conn.request("GET", '/nba/trial/v7/en/league/hierarchy.json?api_key=%s' %API_key)
res = conn.getresponse()
data = res.read()
team_info = data.decode("utf-8")

In [163]:
# add id to team json
def flat(l):
  return [y for x in l for y in x]
# team_info = json.loads(team_info)
divisions = flat([c['divisions'] for c in team_info['conferences']])
flat([t2['teams'] for t2 in divisions])
team_id_dict = {t['market'] + ' ' + t['name']:t['id'] for t in flat([t2['teams'] for t2 in divisions])}
od = collections.OrderedDict(sorted(team_id_dict.items()))

[{'id': '583ec8d4-fb46-11e1-82cb-f4ce4684ea4c',
  'name': 'Wizards',
  'market': 'Washington',
  'alias': 'WAS',
  'sr_id': 'sr:team:3431',
  'reference': '1610612764',
  'venue': {'id': 'f62d5b49-d646-56e9-ba60-a875a00830f8',
   'name': 'Capital One Arena',
   'capacity': 20356,
   'address': '601 F Street NW',
   'city': 'Washington',
   'state': 'DC',
   'zip': '20004',
   'country': 'USA',
   'sr_id': 'sr:venue:6016'}},
 {'id': '583ec97e-fb46-11e1-82cb-f4ce4684ea4c',
  'name': 'Hornets',
  'market': 'Charlotte',
  'alias': 'CHA',
  'sr_id': 'sr:team:3430',
  'reference': '1610612766',
  'venue': {'id': 'a380f011-6e5d-5430-9f37-209e1e8a9b6f',
   'name': 'Spectrum Center',
   'capacity': 19077,
   'address': '330 E. Trade Street',
   'city': 'Charlotte',
   'state': 'NC',
   'zip': '28202',
   'country': 'USA',
   'sr_id': 'sr:venue:6146'}},
 {'id': '583ecb8f-fb46-11e1-82cb-f4ce4684ea4c',
  'name': 'Hawks',
  'market': 'Atlanta',
  'alias': 'ATL',
  'sr_id': 'sr:team:3423',
  'refere

In [139]:
# save team ids in team json
with open(data_folder + 'teams.json', "r") as read_content:
    team_json = json.load(read_content)
for i,team in enumerate(team_json):
    team['sr-id'] = od[list(od.keys())[i]]
with open(data_folder + 'teams.json', 'w') as outfile:
    json.dump(team_json, outfile)

get team rosters --> get sr-ids for each player

In [175]:
team_sr_ids = [t['sr-id'] for t in team_json]
get_conn()
player_sr_id_dict = {}
for t in team_sr_ids:
    time.sleep(1)
    conn.request("GET", "/nba/trial/v7/en/teams/" + t + "/profile.json?api_key=%s" %API_key)
    res = conn.getresponse()
    data = res.read()
    team_profile = json.loads(data.decode("utf-8"))
    player_sr_id_dict.update({t['full_name']:t['id'] for t in team_profile['players']})

In [177]:
# player_sr_id_dict
with open(data_folder + 'players.json', 'w') as outfile:
    json.dump(player_sr_id_dict, outfile)

In [121]:
# for id in season_2022_ids:
# id = season_2022_ids[0]
season = '2022'
team_id = list(team_id_dict.values())[0]
# team_id
# conn.request("GET", "/nba/trial/v7/en/games/"+id+"/boxscore.json?api_key=%s" %API_key)
conn.request("GET", "/nba/trial/v7/en/seasons/" + season + "/REG/teams/" + team_id + "/splits/game.json?api_key=%s" %API_key)
res = conn.getresponse()
data = res.read()
sample_team_splits = data.decode("utf-8")
sample_team_splits = json.loads(sample_team_splits) 

In [ ]:
sample_team_splits['splits'][0].keys()

get player splits for each game (only found totals/averages for season so far)

get player splits for season

In [216]:
get_conn()
# p_id = list(player_sr_id_dict.values())[0]
i = 0
players_splits = {}
for p_name, p_id in player_sr_id_dict.items():
    time.sleep(1)
    if i % 25 == 0:
        print(p_name, p_id)
    conn.request("GET","/nba/trial/v7/en/players/"+p_id+"/profile.json?api_key=%s"%API_key)
    res = conn.getresponse()
    data = res.read()
    player = data.decode("utf-8")
    player = json.loads(player)
    season_stats = player['seasons']
    player_dict = {s['year']: {'team':s['teams'][0]['market'] +' ' + s['teams'][0]['name'], 'total':s['teams'][0]['total'],'average':s['teams'][0]['average'] } for s in season_stats}
    players_splits[p_name] = player_dict
    i= i + 1

Justin Holiday 05dea31d-f1ff-491b-9f17-8be88b26f413
Derrick White 9bcd5cff-4a1c-4454-89b6-a5899b0c6bcc
Kai Jones 116f036d-9233-462b-80bc-3ede69b97706
Malcolm Hill 787f64f8-b127-47da-b05b-6b4324a502a0
Tim Hardaway Jr. 35cd1338-c56b-4247-b53c-264585c59883
Bones Hyland 7ad01370-ae8e-40ae-b70d-61f6d8dba8d9
Patrick Baldwin Jr. 003883ca-cd7e-4b58-99d7-6fef54afdfeb
Josh Christopher 8121242f-45a6-4632-b8ae-f0846a45016a
Ivica Zubac 2da18268-82d4-4b40-a296-7ceddd435d0b
Cole Swider cae1e67f-229e-46c8-851a-571d728ae315
Bam Adebayo 11c303dc-540c-4090-8e0c-76c2fb2c125a
Wesley Matthews 795bcca3-0cca-448c-8198-85c664f76c75
Dereon Seabron 1ad3d938-c2af-4567-b968-95dd79427fa5
Svi Mykhailiuk 8c0929ae-ce3d-47e0-a964-bf6b8d3981ab
Chet Holmgren eb91a4c8-1a8a-46bf-86e6-e16950b67ef6
Matisse Thybulle 8b43c272-c16b-40c7-a4d5-7bbcefeb1816
Greg Brown III 03258eec-a666-4b8f-86da-9bed8e91c39f
KZ Okpala 77dc9e11-c422-4c1b-9545-0f487095891e
Khem Birch 13ef372a-5fd1-4336-9901-12e120a5fbd7
Walker Kessler 7eccbdf9-58a3-

In [218]:
with open(data_folder + 'player_splits.json', 'w') as outfile:
    json.dump(players_splits, outfile)
# players_splits

In [211]:
sample_player.keys()
season_stats = sample_player['seasons']
# season_stats[0]['teams'][0].keys()
# season_stats[0]['teams'][0]['total']
season_stats[0]['teams'][0]['average']
season_stats[2]
{s['year']: {'team':s['teams'][0]['market'] +' ' + s['teams'][0]['name'], 'total':s['teams'][0]['total'],'average':s['teams'][0]['average'] } for s in season_stats}
# [s for s in season_stats['teams']]

{2022: {'team': 'Atlanta Hawks',
  'total': {'games_played': 20,
   'games_started': 0,
   'minutes': 338.0,
   'field_goals_made': 41,
   'field_goals_att': 105,
   'field_goals_pct': 0.39,
   'two_points_made': 16,
   'two_points_att': 33,
   'two_points_pct': 0.485,
   'three_points_made': 25,
   'three_points_att': 72,
   'three_points_pct': 0.347,
   'blocked_att': 3,
   'free_throws_made': 0,
   'free_throws_att': 0,
   'free_throws_pct': 0.0,
   'offensive_rebounds': 1,
   'defensive_rebounds': 18,
   'rebounds': 19,
   'assists': 17,
   'turnovers': 12,
   'assists_turnover_ratio': 1.42,
   'steals': 5,
   'blocks': 7,
   'personal_fouls': 32,
   'tech_fouls': 0,
   'points': 107,
   'flagrant_fouls': 0,
   'ejections': 0,
   'foulouts': 0,
   'true_shooting_att': 105.0,
   'true_shooting_pct': 0.51,
   'efficiency': 79,
   'points_off_turnovers': 18,
   'points_in_paint': 24,
   'points_in_paint_made': 12,
   'points_in_paint_att': 19,
   'points_in_paint_pct': 0.632,
   'effe

Get box scores for weeks games

In [13]:
with open(data_folder + 'teams.json', "r") as read_content:
    team_json = json.load(read_content)
team_json
team_ids = {t['teamName']:t['sr-id'] for t in team_json}
team_ids

{'Atlanta Hawks': '583ecb8f-fb46-11e1-82cb-f4ce4684ea4c',
 'Boston Celtics': '583eccfa-fb46-11e1-82cb-f4ce4684ea4c',
 'Brooklyn Nets': '583ec9d6-fb46-11e1-82cb-f4ce4684ea4c',
 'Charlotte Hornets': '583ec97e-fb46-11e1-82cb-f4ce4684ea4c',
 'Chicago Bulls': '583ec5fd-fb46-11e1-82cb-f4ce4684ea4c',
 'Cleveland Cavaliers': '583ec773-fb46-11e1-82cb-f4ce4684ea4c',
 'Dallas Mavericks': '583ecf50-fb46-11e1-82cb-f4ce4684ea4c',
 'Denver Nuggets': '583ed102-fb46-11e1-82cb-f4ce4684ea4c',
 'Detroit Pistons': '583ec928-fb46-11e1-82cb-f4ce4684ea4c',
 'Golden State Warriors': '583ec825-fb46-11e1-82cb-f4ce4684ea4c',
 'Houston Rockets': '583ecb3a-fb46-11e1-82cb-f4ce4684ea4c',
 'Indiana Pacers': '583ec7cd-fb46-11e1-82cb-f4ce4684ea4c',
 'LA Clippers': '583ecdfb-fb46-11e1-82cb-f4ce4684ea4c',
 'Los Angeles Lakers': '583ecae2-fb46-11e1-82cb-f4ce4684ea4c',
 'Memphis Grizzlies': '583eca88-fb46-11e1-82cb-f4ce4684ea4c',
 'Miami Heat': '583ecea6-fb46-11e1-82cb-f4ce4684ea4c',
 'Milwaukee Bucks': '583ecefd-fb46-11e1-

In [23]:
# for t,i in team_ids.items():
get_conn()
conn.request("GET", "/nba/trial/v7/en/games/" + list(team_ids.values())[0] + "/boxscore.xml?api_key=")

res = conn.getresponse()
data = res.read()
team_profile_data = data.decode("utf-8")

In [24]:
team_profile_data

'<h1>Developer Inactive</h1>'

Get team profile standings

In [26]:
year = '2022'
get_conn()
conn.request("GET", "/nba/trial/v7/en/seasons/" + year + "/REG/standings.json?api_key=%s"%API_key)
res = conn.getresponse()
data = res.read()
team_standings_data = json.loads(data.decode("utf-8"))

In [31]:
# team_standings_data
def flat(l):
  return [y for x in l for y in x]
# team_info = json.loads(team_info)
divisions = flat([c['divisions'] for c in team_standings_data['conferences']])
divisions
flat([t2['teams'] for t2 in divisions])
team_standings_dict = {t['market'] + ' ' + t['name']:t for t in flat([t2['teams'] for t2 in divisions])}
team_standings_dict['New Orleans Pelicans']

{'id': '583ecc9a-fb46-11e1-82cb-f4ce4684ea4c',
 'name': 'Pelicans',
 'market': 'New Orleans',
 'wins': 18,
 'losses': 8,
 'win_pct': 0.692,
 'points_for': 117.62,
 'points_against': 110.62,
 'point_diff': 7.0,
 'sr_id': 'sr:team:5539',
 'reference': '1610612740',
 'games_behind': {'league': 2.5, 'conference': 0.0, 'division': 0.0},
 'streak': {'kind': 'win', 'length': 7},
 'calc_rank': {'div_rank': 1, 'conf_rank': 1},
 'records': [{'record_type': 'atlantic',
   'wins': 2,
   'losses': 1,
   'win_pct': 0.667},
  {'record_type': 'below_500', 'wins': 9, 'losses': 2, 'win_pct': 0.818},
  {'record_type': 'central', 'wins': 3, 'losses': 1, 'win_pct': 0.75},
  {'record_type': 'conference', 'wins': 12, 'losses': 5, 'win_pct': 0.706},
  {'record_type': 'division', 'wins': 5, 'losses': 1, 'win_pct': 0.833},
  {'record_type': 'equal_rebounds', 'wins': 1, 'losses': 0, 'win_pct': 1.0},
  {'record_type': 'equal_turnovers', 'wins': 0, 'losses': 1, 'win_pct': 0.0},
  {'record_type': 'fewer_turnovers',

In [32]:
with open(data_folder + 'team_standings.json', 'w') as outfile:
    json.dump(team_standings_dict, outfile)

Get team splits across Season

In [95]:
year = '2022'
get_conn()
team_splits = {}
split_splits = [0,4,10]

for t,i in team_ids.items():
    print(t)
    time.sleep(1)
    data = get_data("/nba/trial/v7/en/seasons/" + year + "/REG/teams/" + i + "/splits/ingame.json?api_key=%s"%API_key)
    total_games = 0
    weighted_opp_splits_list = []
    weighted_self_splits_list = []
    for s in split_splits:
        split_games = data['splits'][s]['opponents']['total']['games_played']
        total_games = total_games + split_games
        opp_splits = data['splits'][s]['opponents']['average']
        weighted_opp_splits = {s:opp_splits[s] * split_games for s in list(opp_splits.keys())}
        weighted_opp_splits_list.append(weighted_opp_splits)

        self_splits = data['splits'][s]['own_record']['average']
        weighted_self_splits = {s:self_splits[s] * split_games for s in list(self_splits.keys())}
        weighted_self_splits_list.append(weighted_self_splits)

    list_keys = [k for k in list(weighted_opp_splits.keys())]
    total_weighted_opp_splits = dict(zip(list_keys,[0] * len(list_keys)))
    for w in weighted_opp_splits_list:
        for k in list_keys:
            total_weighted_opp_splits[k] += w[k]
    total_weighted_opp_splits = {k:round(i/total_games,2) for k,i in total_weighted_opp_splits.items()}

    total_weighted_self_splits = dict(zip(list_keys,[0] * len(list_keys)))
    for w in weighted_self_splits_list:
        for k in list_keys:
            total_weighted_self_splits[k] += w[k]
    total_weighted_self_splits = {k:round(i/total_games,2) for k,i in total_weighted_self_splits.items()}

    all_splits_dict = {'opponent':total_weighted_opp_splits,'self':total_weighted_self_splits}
    team_splits[t] = all_splits_dict
team_splits

Atlanta Hawks
Boston Celtics
Brooklyn Nets
Charlotte Hornets
Chicago Bulls
Cleveland Cavaliers
Dallas Mavericks
Denver Nuggets
Detroit Pistons
Golden State Warriors
Houston Rockets
Indiana Pacers
LA Clippers
Los Angeles Lakers
Memphis Grizzlies
Miami Heat
Milwaukee Bucks
Minnesota Timberwolves
New Orleans Pelicans
New York Knicks
Oklahoma City Thunder
Orlando Magic
Philadelphia 76ers
Phoenix Suns
Portland Trail Blazers
Sacramento Kings
San Antonio Spurs
Toronto Raptors
Utah Jazz
Washington Wizards


{'Atlanta Hawks': {'opponent': {'fast_break_pts': 13.67,
   'points_off_turnovers': 15.67,
   'second_chance_pts': 14.04,
   'minutes': 242.75,
   'points': 114.12,
   'off_rebounds': 11.07,
   'def_rebounds': 34.63,
   'rebounds': 45.7,
   'assists': 24.53,
   'steals': 7.26,
   'blocks': 4.96,
   'turnovers': 15.0,
   'personal_fouls': 21.04,
   'flagrant_fouls': 0.04,
   'blocked_att': 4.81,
   'field_goals_made': 41.56,
   'field_goals_att': 88.45,
   'three_points_made': 10.82,
   'three_points_att': 32.04,
   'free_throws_made': 20.22,
   'free_throws_att': 25.11,
   'two_points_made': 30.74,
   'two_points_att': 56.41,
   'points_in_paint': 52.89,
   'efficiency': 129.82,
   'true_shooting_att': 99.49,
   'points_in_paint_att': 46.33,
   'points_in_paint_made': 26.44,
   'bench_points': 33.93,
   'fouls_drawn': 19.96,
   'offensive_fouls': 2.67,
   'fast_break_att': 8.33,
   'fast_break_made': 5.23,
   'second_chance_att': 10.41,
   'second_chance_made': 5.15},
  'self': {'fast_

In [ ]:
with open(data_folder + 'team_splits.json', 'w') as outfile:
    json.dump(team_splits, outfile)

In [92]:
[d['category'] for d in data['splits']] # get games with higher fg, same fg, and under fg to get total splits
# data['splits']
split_splits = [0,4,10]
total_games = 0
weighted_opp_splits_list = []
weighted_self_splits_list = []
for s in split_splits:
    split_games = data['splits'][s]['opponents']['total']['games_played']
    total_games = total_games + split_games
    opp_splits = data['splits'][s]['opponents']['average']
    weighted_opp_splits = {s:opp_splits[s] * split_games for s in list(opp_splits.keys())}
    weighted_opp_splits_list.append(weighted_opp_splits)

    self_splits = data['splits'][s]['own_record']['average']
    weighted_self_splits = {s:self_splits[s] * split_games for s in list(self_splits.keys())}
    weighted_self_splits_list.append(weighted_self_splits)

list_keys = [k for k in list(weighted_opp_splits.keys())]
total_weighted_opp_splits = dict(zip(list_keys,[0] * len(list_keys)))
for w in weighted_opp_splits_list:
    for k in list_keys:
        total_weighted_opp_splits[k] += w[k]
total_weighted_opp_splits = {k:round(i/total_games,2) for k,i in total_weighted_opp_splits.items()}

total_weighted_self_splits = dict(zip(list_keys,[0] * len(list_keys)))
for w in weighted_self_splits_list:
    for k in list_keys:
        total_weighted_self_splits[k] += w[k]
total_weighted_self_splits = {k:round(i/total_games,2) for k,i in total_weighted_self_splits.items()}

all_splits_dict = {'opponent':total_weighted_opp_splits,'self':total_weighted_self_splits}
all_splits_dict

{'fast_break_pts': 13.67, 'points_off_turnovers': 15.67, 'second_chance_pts': 14.04, 'minutes': 242.75, 'points': 114.12, 'off_rebounds': 11.07, 'def_rebounds': 34.63, 'rebounds': 45.7, 'assists': 24.53, 'steals': 7.26, 'blocks': 4.96, 'turnovers': 15.0, 'personal_fouls': 21.04, 'flagrant_fouls': 0.04, 'blocked_att': 4.81, 'field_goals_made': 41.56, 'field_goals_att': 88.45, 'three_points_made': 10.82, 'three_points_att': 32.04, 'free_throws_made': 20.22, 'free_throws_att': 25.11, 'two_points_made': 30.74, 'two_points_att': 56.41, 'points_in_paint': 52.89, 'efficiency': 129.82, 'true_shooting_att': 99.49, 'points_in_paint_att': 46.33, 'points_in_paint_made': 26.44, 'bench_points': 33.93, 'fouls_drawn': 19.96, 'offensive_fouls': 2.67, 'fast_break_att': 8.33, 'fast_break_made': 5.23, 'second_chance_att': 10.41, 'second_chance_made': 5.15}


{'opponent': {'fast_break_pts': 13.67,
  'points_off_turnovers': 15.67,
  'second_chance_pts': 14.04,
  'minutes': 242.75,
  'points': 114.12,
  'off_rebounds': 11.07,
  'def_rebounds': 34.63,
  'rebounds': 45.7,
  'assists': 24.53,
  'steals': 7.26,
  'blocks': 4.96,
  'turnovers': 15.0,
  'personal_fouls': 21.04,
  'flagrant_fouls': 0.04,
  'blocked_att': 4.81,
  'field_goals_made': 41.56,
  'field_goals_att': 88.45,
  'three_points_made': 10.82,
  'three_points_att': 32.04,
  'free_throws_made': 20.22,
  'free_throws_att': 25.11,
  'two_points_made': 30.74,
  'two_points_att': 56.41,
  'points_in_paint': 52.89,
  'efficiency': 129.82,
  'true_shooting_att': 99.49,
  'points_in_paint_att': 46.33,
  'points_in_paint_made': 26.44,
  'bench_points': 33.93,
  'fouls_drawn': 19.96,
  'offensive_fouls': 2.67,
  'fast_break_att': 8.33,
  'fast_break_made': 5.23,
  'second_chance_att': 10.41,
  'second_chance_made': 5.15},
 'self': {'fast_break_pts': 13.89,
  'points_off_turnovers': 19.07,
 